# ByteTrack

ByteTrack — это современный алгоритм многопотокового трекинга объектов (Multi-Object Tracking, MOT), который эффективно связывает объекты между кадрами видео, даже если они временно пропадают из-за перекрытий или ложных срабатываний.

Он был представлен в 2021 году как улучшение классического SORT (Simple Online and Realtime Tracking) и DeepSORT, и особенно хорошо работает в сложных сценах с высокой плотностью объектов.

Особенности ByteTrack:

1. Использование "слабых" детекций

* В отличие от SORT/DeepSORT, которые отфильтровывают детекции с низким confidence (например, < 0.5), ByteTrack сохраняет их для сопоставления с "потерянными" треками.

* Это помогает избежать "пропажи" объектов при временных ложных негативах детектора.

2. Двухэтапное сопоставление

* Первая стадия: Сопоставление треков с "надежными" детекциями (высокий confidence).

* Вторая стадия: Сопоставление оставшихся треков с "слабыми" детекциями (низкий confidence).

3. Отслеживание по bounding box (без ReID)

ByteTrack полагается на IoU (Intersection over Union) и движение объектов (через Kalman Filter), но не использует re-identification (ReID) модели, что делает его быстрее DeepSORT.

## Реализация упрощенного трекера ByteTrack

In [ ]:
!pip install numpy opencv-python Pillow onemetric
!pip install git+https://github.com/ifzhang/ByteTrack.git --no-deps

  Cloning https://github.com/ifzhang/ByteTrack.git to /tmp/pip-req-build-ia5t5xmk
  Running command git clone --filter=blob:none --quiet https://github.com/ifzhang/ByteTrack.git /tmp/pip-req-build-ia5t5xmk
  Resolved https://github.com/ifzhang/ByteTrack.git to commit d1bf0191adff59bc8fcfeaa0b33d3d1642552a99
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install ultralytics

In [ ]:
import cv2
import numpy as np
from datetime import datetime
from ultralytics import YOLO

# Своя реализация IoU на NumPy
def iou_batch(bboxes1, bboxes2):
    x11, y11, x12, y12 = np.split(bboxes1, 4, axis=1)
    x21, y21, x22, y22 = np.split(bboxes2, 4, axis=1)

    xA = np.maximum(x11, x21.T)
    yA = np.maximum(y11, y21.T)
    xB = np.minimum(x12, x22.T)
    yB = np.minimum(y12, y22.T)

    interArea = np.maximum(0, xB - xA) * np.maximum(0, yB - yA)
    boxAArea = (x12 - x11) * (y12 - y11)
    boxBArea = (x22 - x21) * (y22 - y21)

    iou = interArea / (boxAArea + boxBArea.T - interArea)
    return iou

class SimpleByteTracker:
    def __init__(self, track_thresh=0.5, match_thresh=0.8, max_misses=5):
        self.track_thresh = track_thresh
        self.match_thresh = match_thresh
        self.max_misses = max_misses  # Макс. число кадров без обновления
        self.tracks = []
        self.next_id = 1

    def update(self, detections, img_size):
        valid_dets = [d for d in detections if d[4] >= self.track_thresh]
        matched = set()
        matched_tracks = set()

        # Увеличиваем счётчик пропусков для всех треков
        for track in self.tracks:
            track['misses'] += 1

        if self.tracks and valid_dets:
            track_boxes = np.array([t['bbox'] for t in self.tracks])
            det_boxes = np.array([d[:4] for d in valid_dets])

            iou_matrix = iou_batch(track_boxes, det_boxes)

            for i, track in enumerate(self.tracks):
                best_match = np.argmax(iou_matrix[i])
                if iou_matrix[i, best_match] > self.match_thresh:
                    track['bbox'] = valid_dets[best_match][:4]
                    track['misses'] = 0  # Сброс счётчика
                    matched.add(best_match)
                    matched_tracks.add(i)

        # Удаляем треки, которые долго не обновлялись
        self.tracks = [
            t for t in self.tracks
            if t['misses'] <= self.max_misses
        ]

        # Добавляем новые треки
        for i, det in enumerate(valid_dets):
            if i not in matched:
                self.tracks.append({
                    'id': self.next_id,
                    'bbox': det[:4],
                    'score': det[4],
                    'misses': 0  # Инициализация счётчика
                })
                self.next_id += 1

        return self.tracks

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
# Инициализация модели YOLOv8
model = YOLO("yolov8n.pt")  # Загрузка nano-модели (можно yolov8s.pt, yolov8m.pt и т.д.)

# Инициализация видео
cap = cv2.VideoCapture('/content/drive/MyDrive/datasets/cv/pedestrian.mp4')  # Или путь к видеофайлу

# Настройки выходного видео
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Создаем имя файла с текущей датой/временем
output_filename = f"tracking_result_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

tracker = SimpleByteTracker()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Детекция объектов (можно настроить confidence threshold)
    results = model(frame, conf=0.5)  # conf - порог уверенности

    # Преобразуем результаты в формат [x1, y1, x2, y2, conf, class_id]
    detections = []
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # bounding boxes
        confs = result.boxes.conf.cpu().numpy()  # confidence scores
        class_ids = result.boxes.cls.cpu().numpy()  # class IDs

        for box, conf, cls_id in zip(boxes, confs, class_ids):
            x1, y1, x2, y2 = box
            detections.append([x1, y1, x2, y2, conf, cls_id])


    # Обновление трекера
    tracks = tracker.update(detections, (frame.shape[1], frame.shape[0]))

    # Рисуем результат на кадре
    for track in tracks:
        x1, y1, x2, y2 = map(int, track['bbox'])
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track['id']}", (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Записываем кадр в видео
    out.write(frame)

# Освобождаем ресурсы
cap.release()
out.release()
# cv2.destroyAllWindows()

print(f"Видео сохранено как: {output_filename}")


0: 384x640 2 persons, 10 cars, 9.7ms
Speed: 2.1ms preprocess, 9.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 10 cars, 8.5ms
Speed: 2.2ms preprocess, 8.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 11 cars, 6.9ms
Speed: 2.1ms preprocess, 6.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 13 cars, 7.4ms
Speed: 2.1ms preprocess, 7.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 10 cars, 10.4ms
Speed: 2.1ms preprocess, 10.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 10 cars, 7.4ms
Speed: 2.0ms preprocess, 7.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 10 cars, 10.0ms
Speed: 2.1ms preprocess, 10.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 10 cars, 7.4ms
Speed: 2.0ms preprocess, 

## Используя код выше реализуйте задания ниже (все в одной общей задаче)

## Задача 1

Сделайте так, чтобы трекер работал только в заданной области (например, внутри прямоугольника [x1, y1, x2, y2]).

## Задача 2

Сделайте так, чтобы цвет bbox менялся в зависимости от misses

## Задача 3

Добавьте трекам поле history, хранящее последние N координат, и визуализируйте их с ниспадающей яркостью

## Задача 4

Выводите для кадра:
1. Число активных треков.
2. Среднюю скорость движения (пикселей/кадр) для каждого объекта.